In [13]:
from apscheduler.schedulers.background import BlockingScheduler
from utils.event import EventEngine, Event
from utils.brokers import Broker
from gateway.binance_http import Interval
from constant.constant import EVENT_KLINE, EVENT_POS, EVENT_OPEN_ORDERS
import pandas as pd
from strategies.boll_macd_strategy import BollMacdStrategy
import pandas_ta as ta
import numpy as np
import logging
from pandas_ta.utils import get_offset, verify_series
from decimal import Decimal
from datetime import datetime, timedelta
from transitions import Machine
from technical.indicators import SSLChannels

In [14]:
def get_kline_data(symbol: str, interval: Interval):
    data = broker.binance_http.get_kline(symbol=symbol, interval=interval, limit=100)

    df = pd.DataFrame(data, columns={"open_time": 0, "open": 1, "high": 2, "low": 3, "close": 4, "volume": 5, "close_time": 6,
                               "trade_money": 7, "trade_count": 8, "buy_volume": 9, "sell_volume": 10, "other": 11})
    df = df[["open_time", "open", "high", "low", "close", "volume", "close_time"]]
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms') + pd.Timedelta(hours=8)
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms') + pd.Timedelta(hours=8)
    return df

In [15]:
def calHa(df):
    dataframe = df.copy()
    dataframe["open"]=df['open'].astype('float')
    dataframe["high"]=df['high'].astype('float')
    dataframe["low"]=df['low'].astype('float')
    dataframe["close"]=df['close'].astype('float')

    ha = ta.ha(dataframe.open, dataframe.high, dataframe.low, dataframe.close)
    ha["volume"] = df['volume']
    return ha

In [16]:
key = "t3uT9bnVRXYX6OIAH2MSynTTCvHj8z9aOq1EPC70ugEMBYzyoUSko1meWFR9QJrF"
secret = '3VwOIUYNTL6So9H7Epmody6pjBj32MOazcb9WLGb384C950C65ij0wGYk8dOjOpN'

engine = EventEngine()
broker = Broker(engine, key=key, secret=secret)

data = get_kline_data('ETHUSDT', Interval.MINUTE_3)
df = data[["open_time", "open", "high", "low", "close", "volume"]]
df

,open_time,open,high,low,close,volume
0,2022-06-04 18:15:00,1774.66,1777.49,1774.66,1776.74,3103.830
1,2022-06-04 18:18:00,1776.74,1776.75,1775.09,1776.09,1972.847
2,2022-06-04 18:21:00,1776.10,1776.37,1774.28,1774.31,1832.017
3,2022-06-04 18:24:00,1774.32,1775.42,1774.12,1774.55,1234.107
4,2022-06-04 18:27:00,1774.56,1775.06,1771.58,1771.96,3798.197
...,...,...,...,...,...,...
95,2022-06-04 23:00:00,1776.16,1778.28,1774.48,1775.09,4587.336
96,2022-06-04 23:03:00,1775.09,1775.42,1773.35,1774.06,2919.866
97,2022-06-04 23:06:00,1774.06,1775.72,1774.06,1774.63,2059.589
98,2022-06-04 23:09:00,1774.64,1779.24,1774.64,1778.58,4205.889


websocket open..


In [17]:
#计算HA
ha = calHa(df)
print(ha)
ha["open_time"] = df["open_time"]

        HA_open      HA_high       HA_low   HA_close    volume
0   1775.700000  1777.490000  1774.660000  1775.8875  3103.830
1   1775.793750  1776.750000  1775.090000  1776.1675  1972.847
2   1775.980625  1776.370000  1774.280000  1775.2650  1832.017
3   1775.622813  1775.622813  1774.120000  1774.6025  1234.107
4   1775.112656  1775.112656  1771.580000  1773.2900  3798.197
..          ...          ...          ...        ...       ...
95  1776.454574  1778.280000  1774.480000  1776.0025  4587.336
96  1776.228537  1776.228537  1773.350000  1774.4800  2919.866
97  1775.354269  1775.720000  1774.060000  1774.6175  2059.589
98  1774.985884  1779.240000  1774.640000  1776.7750  4205.889
99  1775.880442  1778.940000  1775.880442  1778.3975  2629.881

[100 rows x 5 columns]


In [18]:
dataframe = pd.DataFrame()
dataframe["close"] = ha["HA_close"].astype('float')
dataframe["high"] = ha["HA_high"].astype('float')
dataframe["low"] = ha["HA_low"].astype('float')
dataframe["open_time"] = ha["open_time"]

ssl = SSLChannels(dataframe)
print(ssl)

        close         high          low           open_time      smaHigh       smaLow  hlv      sslDown        sslUp
0   1775.8875  1777.490000  1774.660000 2022-06-04 18:15:00          NaN          NaN  NaN          NaN          NaN
1   1776.1675  1776.750000  1775.090000 2022-06-04 18:18:00          NaN          NaN  NaN          NaN          NaN
2   1775.2650  1776.370000  1774.280000 2022-06-04 18:21:00          NaN          NaN  NaN          NaN          NaN
3   1774.6025  1775.622813  1774.120000 2022-06-04 18:24:00          NaN          NaN  NaN          NaN          NaN
4   1773.2900  1775.112656  1771.580000 2022-06-04 18:27:00          NaN          NaN  NaN          NaN          NaN
..        ...          ...          ...                 ...          ...          ...  ...          ...          ...
95  1776.0025  1778.280000  1774.480000 2022-06-04 23:00:00  1780.580465  1775.463588  1.0  1775.463588  1780.580465
96  1774.4800  1776.228537  1773.350000 2022-06-04 23:03:00  177

In [21]:
sslsslDown1 = ssl["sslDown"][97]
sslsslUp1 = ssl["sslUp"][97]
sslsslDown2 = ssl["sslDown"][98]
sslsslUp2 = ssl["sslUp"][98]

Conditon4 = 0 #上上根K line
Conditon5 = 0 #上根K line
if sslsslUp1 > sslsslDown1:
    Conditon4 = 1
else:
    Conditon4 = -1

if sslsslUp2 > sslsslDown2:
    Conditon5 = 1
else:
    Conditon5 = -1

Conditon2 = 0 
if (Conditon4 == -1) and (Conditon5 == 1):
    Conditon2 = 1
if (Conditon4 == 1) and (Conditon5 == -1):
    Conditon2 = -1

print("Conditon4: ", Conditon4)
print("Conditon5: ", Conditon5)
print("Conditon2: ", Conditon2)


1778.832518556667

In [ ]:
period=input(title="Period", defval=10)
len=input(title="Period", defval=10)
smaHigh=sma(high, len)
smaLow=sma(low, len)
Hlv = na
Hlv := close > smaHigh ? 1 : close < smaLow ? -1 : Hlv[1]
sslDown = Hlv < 0 ? smaHigh: smaLow
sslUp   = Hlv < 0 ? smaLow : smaHigh

plot(sslDown, linewidth=2, color=red)
plot(sslUp, linewidth=2, color=lime)